In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from zipfile import ZipFile
from PIL import Image,ImageEnhance
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pylab as plt


In [ ]:
path = '/kaggle/input/dogs-vs-cats/'

with ZipFile('' + path + 'train.zip', 'r') as train_zip:
    train_zip.extractall('./')

with ZipFile('' + path + 'test1.zip', 'r') as test_zip:
    test_zip.extractall('./')

In [ ]:
img_size = 256
batch_size=16
epochs=10

# Data Generator

In [ ]:
path_train = './train/'
train_data_name = os.listdir(path_train)
train_data_lable = []#np.zeros((np.size(train_data_name),1),dtype="int")

for i in range(0,np.size(train_data_name)):
 if train_data_name[i].rsplit('.')[0] == 'cat':
    train_data_lable.append("cat")
 else:
    train_data_lable.append("dog")

train_data_df = pd.DataFrame({
    'train_data_name': train_data_name,
    'train_data_lable': train_data_lable
})
#train_data_df['train_data_lable']=train_data_df['train_data_lable'].astype('str')
train_data_df.head()

In [ ]:
path_test = './test1/'
test_data_name = os.listdir(path_test)
test_ID = []
for i in range(0,np.size(test_data_name)):
    test_ID.append(data_name[i].rsplit('.')[1])
test_data_df = pd.DataFrame({
    'test_data_name': test_data_name
})
test_data_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_valid_df, test_df = train_test_split(train_data_df, test_size = 0.04)
train_df, valid_df = train_test_split(train_valid_df, test_size = 0.2)

In [ ]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                                                              #rescale=1./255,
                                                              #shear_range=0.2,
                                                              #zoom_range=0.2,
                                                              horizontal_flip=True,
                                                              width_shift_range=0.2,
                                                              height_shift_range=0.2,
                                                              fill_mode='nearest',
                                                              preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    path_train, 
                                                    x_col='train_data_name',
                                                    y_col='train_data_lable',
                                                    class_mode='binary',
                                                    target_size=(img_size, img_size),
                                                    batch_size=batch_size
)

In [ ]:
valid_datagen=tf.keras.preprocessing.image.ImageDataGenerator(#rescale=1./255,
                                                              preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)
valid_generator = train_datagen.flow_from_dataframe(
    valid_df,
    path_train, 
    x_col='train_data_name',
    y_col='train_data_lable',
    class_mode='binary',
    target_size=(img_size, img_size),
    batch_size=batch_size
)

In [ ]:
class_map = {v: k for k, v in train_generator.class_indices.items()}
plt.figure(figsize=(8, 8))
batch = next(train_generator)
for x in range(0,9):
    plt.subplot(3, 3, x+1)
    plt.imshow(batch[0][x], interpolation='nearest')
    item_label = batch[1][x]
    item_label = class_map[int(item_label)]
    plt.title(item_label)
    plt.axis('off')

plt.show()

# ResNet

In [ ]:
resNet = tf.keras.applications.ResNet50V2(weights = 'imagenet',include_top = False,input_shape = (img_size,img_size, 3))

tf.keras.trainable=False # Freeze layers
resNet_model = tf.keras.Sequential([
        resNet,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation = 'relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')])
resNet_model.summary()

In [ ]:
loss_function='binary_crossentropy'
filepath='./'
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=0,mode='min')
ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath + 'Resnet50_best.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only = True)
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
callback=[EarlyStopping,ReduceLROnPlateau,ModelCheckpoint]

In [ ]:
resNet_model.compile(optimizer=optimizer,loss=loss_function,metrics=['accuracy'])

In [ ]:
resnet_his = resNet_model.fit(train_generator,batch_size=batch_size,epochs=15,validation_data=valid_generator,steps_per_epoch=15,validation_steps=15,callbacks=callback)

In [ ]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Steps")
plt.ylim([0,1])
plt.plot(resnet_his.history["accuracy"])
plt.plot(resnet_his.history["val_accuracy"])
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Steps")
plt.ylim([0,1])
plt.plot(resnet_his.history['loss'],) 
plt.plot(resnet_his.history['val_loss'],)



In [ ]:
test_aug_map = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

test_df = test_aug_map.flow_from_dataframe(
    test_df, path_train,
    x_col='train_data_name',
    y_col='train_data_lable',
    class_mode = 'binary',
    target_size = (img_size, img_size))

In [ ]:
test_aug_map = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

test_data = test_aug_map.flow_from_dataframe(
    test_data_df, path_test,
    x_col='test_data_name',
    y_col=None,
    class_mode = None,
    target_size = (img_size, img_size),
    shuffle = False)


In [ ]:
resNet_model.evaluate(test_df)

In [ ]:
test_data.reset()
test_label = resNet_model.predict(test_data,verbose=1)

In [ ]:
test_label_c = [1 if l[0] > 0.5 else 0 for l in test_label]

In [ ]:
test_label_show = []
for l in test_label:
    if l[0] < 0.5:
        test_label_show.append('cat')
    else:
        test_label_show.append('dog')

In [ ]:
n_to_show = 10
indices = np.random.choice(range(len(test_ID)), n_to_show)
fig = plt.figure(figsize=(20, 20))
fig.subplots_adjust(hspace=0.4, wspace=0.2)
for i, idx in enumerate(indices):
    img = Image.open(path_test+test_ID[idx]+".jpg")
    a = 1
    b=i+1
    if i > 4:
        a=2
        b = b-5
    ax = fig.add_subplot(a, n_to_show, b)
    ax.axis('off')
    ax.text(0.5, -0.35, 'pred = ' + str(test_label_show[idx]), fontsize=10, ha='center', transform=ax.transAxes)
    ax.imshow(img)

In [ ]:
output =pd.DataFrame({'id': test_ID,'label': test_label_c})
output.head
output.to_csv('submission.csv', index=False)